In [1]:
import pandas as pd
import numpy as np

pd.set_option('max_columns', 100)
pd.set_option('max_rows', 200)
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# 验证集

In [2]:
train = pd.read_csv('train_all.csv')

In [3]:
train_col = train.columns
for i in train_col:
    train[i] = train[i].replace('\\N', 0)

In [4]:
train['former_complaint_fee'] = train['former_complaint_fee']/1000

In [5]:
train['2_total_fee'] = train['2_total_fee'].astype(float)
train['3_total_fee'] = train['3_total_fee'].astype(float)

In [6]:
train = train.drop(train[train['contract_type']==8].index.tolist())

In [1164]:
# train['month_traffic'] = np.log(train['month_traffic'])
# train['service2_callser_time'] = np.log(train['service2_caller_time'])

In [765]:
# test['2_total_fee'] = test['2_total_fee'].astype(float)
# test['3_total_fee'] = test['3_total_fee'].astype(float)

涨分

In [7]:
# 其中fee_min有没有用还没有确定
train['fee_mean'] = train.loc[:, '1_total_fee':'4_total_fee'].mean(axis=1)
# train['fee_median'] = train.loc[:, '1_total_fee':'4_total_fee'].median(axis=1)
train['fee_max'] = train.loc[:, '1_total_fee':'4_total_fee'].max(axis=1)
train['fee_min'] = train.loc[:, '1_total_fee':'4_total_fee'].min(axis=1)
# train['fee_var'] = train.loc[:, '1_total_fee':'4_total_fee'].var(axis=1)

In [912]:
# train['every_complaint_fee'] = 0
# index_not0 = train[train['former_complaint_fee'] != 0].index.tolist()
# train['every_complaint_fee'][index_not0] = train['former_complaint_fee'][index_not0]/train['former_complaint_num'][index_not0]

In [768]:
# train['online_morethan_contract'] = (train['online_time']>train['contract_time']).astype(int)
# train['local_morethan_service1'] = (train['local_caller_time']>train['service1_caller_time']).astype(int)
# train['local_morethan_serivce2'] = (train['local_caller_time']>train['service2_caller_time']).astype(int)
# train['service1_morethan_service2'] = (train['service1_caller_time']>train['service2_caller_time']).astype(int)

In [8]:
train['every_pay'] = train['pay_num']/train['pay_times']

In [9]:
train['this_month_traffic'] = train['month_traffic'] - train['last_month_traffic']

In [1213]:
train['actual_1_total_fee'] = train['1_total_fee'] - train['service1_caller_time']*0.15-(train['month_traffic']-train['last_month_traffic']-train['local_trafffic_month'])*0.3

In [1214]:
train['total_fee1'] = train['1_total_fee']/train['1_total_fee'].max()
train['total_fee2'] = train['2_total_fee']/train['2_total_fee'].max()
train['total_fee3'] = train['3_total_fee']/train['3_total_fee'].max()
train['total_fee4'] = train['4_total_fee']/train['4_total_fee'].max()

In [1215]:
# train['total_min2'] = train['2_total_fee']/train['2_total_fee'].min()
train['total_min3'] = train['3_total_fee']/train['3_total_fee'].min()
train['total_min4'] = train['4_total_fee']/train['4_total_fee'].min()

## onehot

In [1124]:
# service_type = pd.get_dummies(train['service_type'], prefix='service_type')
# service_type['user_id'] = train['user_id']

In [1216]:
is_mix_service = pd.get_dummies(train['is_mix_service'], prefix='is_mix_service')
is_mix_service['user_id'] = train['user_id']

In [1217]:
many_over_bill = pd.get_dummies(train['many_over_bill'], prefix='many_over_bill')
many_over_bill['user_id'] = train['user_id']

In [1218]:
contract_type = pd.get_dummies(train['contract_type'], prefix='contract_type')
contract_type['user_id'] = train['user_id']

In [1219]:
is_promise_low_consume = pd.get_dummies(train['is_promise_low_consume'], prefix='low_consume')
is_promise_low_consume['user_id'] = train['user_id']

In [1220]:
net_service = pd.get_dummies(train['net_service'], prefix='low_consume')
net_service['user_id'] = train['user_id']

In [1221]:
complaint_level = pd.get_dummies(train['complaint_level'], prefix='complaint_level')
complaint_level['user_id'] = train['user_id']

In [1222]:
onehot = pd.merge(is_mix_service, many_over_bill, on=['user_id'])
# onehot = pd.merge(onehot, many_over_bill, on=['user_id'])
onehot = pd.merge(onehot, contract_type, on=['user_id'])
onehot = pd.merge(onehot, is_promise_low_consume, on=['user_id'])
onehot = pd.merge(onehot, net_service, on=['user_id'])
onehot = pd.merge(onehot, complaint_level, on=['user_id'])

In [1223]:
train = pd.merge(train, onehot, on=['user_id'])

In [1224]:
train = train.drop(['is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume', 'net_service', 'complaint_level'], axis=1)

In [1225]:
train = train.drop(['former_complaint_fee', 'former_complaint_num'], axis=1)

In [8]:
# train['pay_diff_fee'] = train['1_total_fee'] - train['pay_num']

In [1087]:
# train['contract_online'] = train['contract_time'] + train['online_time']

涨分

In [65]:
# train['net_contract'] = train['contract_type'].map(str)+'_'+train['is_mix_service'].map(str)
# train['net_contract'] = pd.factorize(train['net_contract'])[0]

In [66]:
# train['service_contract'] = train['service_type'].map(str)+'_'+train['is_mix_service'].map(str)
# train['service_contract'] = pd.factorize(train['service_contract'])[0]

涨分

In [605]:
# train['online_fee'] = train['fee_mean'] * train['online_time']
# train['online_min_fee'] = train['fee_min'] * train['online_time']
# train['online_max_fee'] = train['fee_max'] * train['online_time']

### 计算total_fee异常值判断的上下界 涨分

In [133]:
# fee_mean = train[['user_id']]
# fee_mean['fee_mean'] = train.loc[:, '1_total_fee':'4_total_fee'].mean(axis=1)
# fee_mean['1_fee_diff'] = (train['1_total_fee'] - fee_mean['fee_mean'])/fee_mean['fee_mean']
# fee_mean['2_fee_diff'] = (train['2_total_fee'] - fee_mean['fee_mean'])/fee_mean['fee_mean']
# fee_mean['3_fee_diff'] = (train['3_total_fee'] - fee_mean['fee_mean'])/fee_mean['fee_mean']
# fee_mean['4_fee_diff'] = (train['4_total_fee'] - fee_mean['fee_mean'])/fee_mean['fee_mean']

In [975]:
fee_mean = train[['user_id', '1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']]

In [976]:
fee_mean = fee_mean.sort_values(by='1_total_fee', axis=0, ascending=True).reset_index(drop=True)
fee_u1 = fee_mean.loc[int(len(fee_mean['1_total_fee'])*3/4), '1_total_fee']
fee_l1 = fee_mean.loc[int(len(fee_mean['1_total_fee'])/4), '1_total_fee']
iqr1 = fee_u1-fee_l1
upper_bound1 = fee_u1 + 1.5*iqr1
lower_bound1 = fee_l1 - 1.5*iqr1

In [977]:
fee_mean = fee_mean.sort_values(by='2_total_fee', axis=0, ascending=True).reset_index(drop=True)
fee_u2 = fee_mean.loc[int(len(fee_mean['2_total_fee'])*3/4), '2_total_fee']
fee_l2 = fee_mean.loc[int(len(fee_mean['2_total_fee'])/4), '2_total_fee']
iqr2 = fee_u2-fee_l2
upper_bound2 = fee_u2 + 1.5*iqr2
lower_bound2 = fee_l2 - 1.5*iqr2

In [978]:
fee_mean3 = fee_mean.sort_values(by='3_total_fee', axis=0, ascending=True).reset_index(drop=True)
fee_u3 = fee_mean.loc[int(len(fee_mean['3_total_fee'])*3/4), '3_total_fee']
fee_l3 = fee_mean.loc[int(len(fee_mean['3_total_fee'])/4), '3_total_fee']
iqr3 = fee_u3-fee_l3
upper_bound3 = fee_u3 + 1.5*iqr3
lower_bound3 = fee_l3 - 1.5*iqr3

In [979]:
fee_mean = fee_mean.sort_values(by='4_total_fee', axis=0, ascending=True).reset_index(drop=True)
fee_u4 = fee_mean.loc[int(len(fee_mean['4_total_fee'])*3/4), '4_total_fee']
fee_l4 = fee_mean.loc[int(len(fee_mean['4_total_fee'])/4), '4_total_fee']
iqr4 = fee_u4-fee_l4
upper_bound4 = fee_u4 + 1.5*iqr4
lower_bound4 = fee_l4 - 1.5*iqr4

In [980]:
fee1_upout = fee_mean[fee_mean['1_total_fee']>upper_bound1].index.tolist()
fee1_lowout = fee_mean[fee_mean['1_total_fee']<lower_bound1].index.tolist()
fee2_upout = fee_mean[fee_mean['2_total_fee']>upper_bound2].index.tolist()
fee2_lowout = fee_mean[fee_mean['2_total_fee']<lower_bound2].index.tolist()
fee3_upout = fee_mean[fee_mean['3_total_fee']>upper_bound3].index.tolist()
fee3_lowout = fee_mean[fee_mean['3_total_fee']<lower_bound3].index.tolist()
fee4_upout = fee_mean[fee_mean['4_total_fee']>upper_bound4].index.tolist()
fee4_lowout = fee_mean[fee_mean['4_total_fee']<lower_bound4].index.tolist()

In [1038]:
train['1_out_fee'] = 0
train['2_out_fee'] = 0
train['3_out_fee'] = 0
train['4_out_fee'] = 0
train['1_out_fee'][fee1_upout] = 1
train['2_out_fee'][fee2_upout] = 1
train['3_out_fee'][fee3_upout] = 1
train['4_out_fee'][fee4_upout] = 1
train['1_out_fee'][fee1_lowout] = -1
train['2_out_fee'][fee2_lowout] = -1
train['3_out_fee'][fee3_lowout] = -1
train['4_out_fee'][fee4_lowout] = -1

In [10]:
train.to_csv('train_pre.csv', index=False)

In [162]:
# train['this_month_traffic'] = train['month_traffic'] - train['last_month_traffic']
# train['outland_traffic_month'] = train['this_month_traffic'] - train['local_trafffic_month']
# train['is_0'] = (train['month_traffic'] 
#                          == 0)&(train['last_month_traffic'] == 0)&(train['local_trafffic_month']==0)

In [163]:
# train['no_call'] = ((train['local_caller_time'] == 0)&
#                            (train['service1_caller_time'] == 0)&
#                            (train['service2_caller_time'] == 0)).astype(int)
# train['no_service_but_call'] = ((train['local_caller_time'] == 0)&
#                                         (train['service1_caller_time'] > 0)|
#                                         (train['service2_caller_time'] > 0)).astype(int)
# train['no_traffic'] = (train['last_month_traffic'] == train['month_traffic']).astype(int)

In [164]:
# train['out_traffic'] = train['month_traffic'] - train['local_trafffic_month']
# train['last_no_out'] = ((train['last_month_traffic'] > 0)& (train['service2_caller_time'] == 0)).astype(int)
# train['last_no_change_type'] = (train['last_month_traffic'] > 0).astype(int)
# train['now_right_fee'] = train['last_no_change_type']*train['2_total_fee']
# del train['last_no_change_type']

In [202]:
# 这一组是一起试的，如果最后有时间，可以分开看一下
# train['remain_fee'] = train['1_total_fee'] - train['month_traffic']*0.3
# train['base_fee'] = (train['month_traffic']-train['last_month_traffic'])*0.3+train['local_caller_time']*0.15
# train['last_remain_fee'] = train['2_total_fee'] - train['service1_caller_time']*0.15
# train['last2_remain_fee'] = train['3_total_fee'] - train['service2_caller_time']*0.15

In [178]:
# train['12_fee'] = train['1_total_fee'] - train['2_total_fee']
# train['23_fee'] = train['2_total_fee'] - train['3_total_fee']
# train['34_fee'] = train['3_total_fee'] - train['4_total_fee']

In [181]:
# train['fee_to1'] = (train['1_total_fee']-train['fee_mean'])/train['fee_var']
# train['fee_to2'] = (train['2_total_fee']-train['fee_mean'])/train['fee_var']
# train['fee_to3'] = (train['3_total_fee']-train['fee_mean'])/train['fee_var']
# train['fee_to4'] = (train['4_total_fee']-train['fee_mean'])/train['fee_var']

In [109]:
# train['1_total_fee'] = train['1_total_fee'].apply(lambda x: np.sqrt(x))
# train['2_total_fee'] = train['2_total_fee'].apply(lambda x: np.sqrt(x))
# train['3_total_fee'] = train['3_total_fee'].apply(lambda x: np.sqrt(x))
# train['4_total_fee'] = train['4_total_fee'].apply(lambda x: np.sqrt(x))

# 测试集

In [1227]:
test = pd.read_csv('republish_test.csv')
# test = pd.read_csv('test_pre_0.743.csv')

In [1228]:
test_col = test.columns
for i in test_col:
    test[i] = test[i].replace('\\N', 0)

In [1229]:
test['2_total_fee'] = test['2_total_fee'].astype(float)
test['3_total_fee'] = test['3_total_fee'].astype(float)

In [1230]:
# test['month_traffic'] = np.log(test['month_traffic'])
# test['service2_callser_time'] = np.log(test['service2_caller_time'])

In [1231]:
test['fee_mean'] = test.loc[:, '1_total_fee':'4_total_fee'].mean(axis=1)
test['fee_median'] = test.loc[:, '1_total_fee':'4_total_fee'].median(axis=1)
test['fee_max'] = test.loc[:, '1_total_fee':'4_total_fee'].max(axis=1)
test['fee_min'] = test.loc[:, '1_total_fee':'4_total_fee'].min(axis=1)
# test['fee_var'] = test.loc[:, '1_total_fee':'4_total_fee'].var(axis=1)

In [988]:
# test['every_complaint_fee'] = 0
# index_not0 = test[test['former_complaint_fee'] != 0].index.tolist()
# test['every_complaint_fee'][index_not0] = test['former_complaint_fee'][index_not0]/test['former_complaint_num'][index_not0]

In [989]:
# test['online_morethan_contract'] = (test['online_time']>test['contract_time']).astype(int)
# test['local_morethan_service1'] = (test['local_caller_time']>test['service1_caller_time']).astype(int)
# test['local_morethan_serivce2'] = (test['local_caller_time']>test['service2_caller_time']).astype(int)
# test['service1_morethan_service2'] = (test['service1_caller_time']>test['service2_caller_time']).astype(int)

In [1232]:
test['every_pay'] = test['pay_num']/test['pay_times']

In [1233]:
test['this_month_traffic'] = test['month_traffic'] - test['last_month_traffic']

In [1234]:
test['actual_1_total_fee'] = test['1_total_fee'] - test['service1_caller_time']*0.15-(test['month_traffic']-test['last_month_traffic']-test['local_trafffic_month'])*0.3

In [1235]:
test['total_fee1'] = test['1_total_fee']/test['1_total_fee'].max()
test['total_fee2'] = test['2_total_fee']/test['2_total_fee'].max()
test['total_fee3'] = test['3_total_fee']/test['3_total_fee'].max()
test['total_fee4'] = test['4_total_fee']/test['4_total_fee'].max()

In [1236]:
# test['total_min2'] = test['2_total_fee']/test['2_total_fee'].min()
test['total_min3'] = test['3_total_fee']/test['3_total_fee'].min()
test['total_min4'] = test['4_total_fee']/test['4_total_fee'].min()

## Onehot

In [1237]:
# service_type = pd.get_dummies(test['service_type'], prefix='service_type')
# service_type['user_id'] = test['user_id']

In [1238]:
is_mix_service = pd.get_dummies(test['is_mix_service'], prefix='is_mix_service')
is_mix_service['user_id'] = test['user_id']

In [1239]:
many_over_bill = pd.get_dummies(test['many_over_bill'], prefix='many_over_bill')
many_over_bill['user_id'] = test['user_id']

In [1240]:
contract_type = pd.get_dummies(test['contract_type'], prefix='contract_type')
contract_type['user_id'] = test['user_id']

In [1241]:
is_promise_low_consume = pd.get_dummies(test['is_promise_low_consume'], prefix='low_consume')
is_promise_low_consume['user_id'] = test['user_id']

In [1242]:
net_service = pd.get_dummies(test['net_service'], prefix='low_consume')
net_service['user_id'] = test['user_id']

In [1243]:
complaint_level = pd.get_dummies(test['complaint_level'], prefix='complaint_level')
complaint_level['user_id'] = test['user_id']

In [1244]:
onehot = pd.merge(is_mix_service, many_over_bill, on=['user_id'])
# onehot = pd.merge(onehot, many_over_bill, on=['user_id'])
onehot = pd.merge(onehot, contract_type, on=['user_id'])
onehot = pd.merge(onehot, is_promise_low_consume, on=['user_id'])
onehot = pd.merge(onehot, net_service, on=['user_id'])
onehot = pd.merge(onehot, complaint_level, on=['user_id'])

In [1245]:
test = pd.merge(test, onehot, on=['user_id'])

In [1246]:
test = test.drop(['is_mix_service', 'many_over_bill', 'contract_type', 'is_promise_low_consume', 'net_service', 'complaint_level'], axis=1)

In [16]:
# test['pay_diff_fee'] = test['1_total_fee'] - test['pay_num']

In [1109]:
# test['contract_online'] = test['contract_time'] + test['online_time']

In [75]:
# test['net_contract'] = test['contract_type'].map(str)+'_'+test['is_mix_service'].map(str)
# test['net_contract'] = pd.factorize(test['net_contract'])[0]

In [76]:
# test['service_contract'] = test['service_type'].map(str)+'_'+test['is_mix_service'].map(str)
# test['service_contract'] = pd.factorize(test['service_contract'])[0]

In [531]:
# test['online_fee'] = test['fee_mean'] * test['online_time']
# test['online_min_fee'] = test['fee_min'] * test['online_time']
# test['online_max_fee'] = test['fee_max'] * test['online_time']

In [1005]:
fee_mean = test[['user_id', '1_total_fee', '2_total_fee', '3_total_fee', '4_total_fee']]

In [1006]:
fee_u1 = fee_mean.loc[int(len(fee_mean['1_total_fee'])*3/4), '1_total_fee']
fee_l1 = fee_mean.loc[int(len(fee_mean['1_total_fee'])/4), '1_total_fee']
iqr1 = fee_u1-fee_l1
upper_bound1 = fee_u1 + 1.5*iqr1
lower_bound1 = fee_l1 - 1.5*iqr1

In [1007]:
fee_u2 = fee_mean.loc[int(len(fee_mean['2_total_fee'])*3/4), '2_total_fee']
fee_l2 = fee_mean.loc[int(len(fee_mean['2_total_fee'])/4), '2_total_fee']
iqr2 = fee_u2-fee_l2
upper_bound2 = fee_u2 + 1.5*iqr2
lower_bound2 = fee_l2 - 1.5*iqr2

In [1008]:
fee_u3 = fee_mean.loc[int(len(fee_mean['3_total_fee'])*3/4), '3_total_fee']
fee_l3 = fee_mean.loc[int(len(fee_mean['3_total_fee'])/4), '3_total_fee']
iqr3 = fee_u3-fee_l3
upper_bound3 = fee_u3 + 1.5*iqr3
lower_bound3 = fee_l3 - 1.5*iqr3

In [1009]:
fee_u4 = fee_mean.loc[int(len(fee_mean['4_total_fee'])*3/4), '4_total_fee']
fee_l4 = fee_mean.loc[int(len(fee_mean['4_total_fee'])/4), '4_total_fee']
iqr4 = fee_u4-fee_l4
upper_bound4 = fee_u4 + 1.5*iqr4
lower_bound4 = fee_l4 - 1.5*iqr4

In [1010]:
fee1_upout = fee_mean[fee_mean['1_total_fee']>upper_bound1].index.tolist()
fee1_lowout = fee_mean[fee_mean['1_total_fee']<lower_bound1].index.tolist()
fee2_upout = fee_mean[fee_mean['2_total_fee']>upper_bound2].index.tolist()
fee2_lowout = fee_mean[fee_mean['2_total_fee']<lower_bound2].index.tolist()
fee3_upout = fee_mean[fee_mean['3_total_fee']>upper_bound3].index.tolist()
fee3_lowout = fee_mean[fee_mean['3_total_fee']<lower_bound3].index.tolist()
fee4_upout = fee_mean[fee_mean['4_total_fee']>upper_bound4].index.tolist()
fee4_lowout = fee_mean[fee_mean['4_total_fee']<lower_bound4].index.tolist()

In [1060]:
test['1_out_fee'] = 0
test['2_out_fee'] = 0
test['3_out_fee'] = 0
test['4_out_fee'] = 0
test['1_out_fee'][fee1_upout] = 1
test['2_out_fee'][fee2_upout] = 1
test['3_out_fee'][fee3_upout] = 1
test['4_out_fee'][fee4_upout] = 1
test['1_out_fee'][fee1_lowout] = -1
test['2_out_fee'][fee2_lowout] = -1
test['3_out_fee'][fee3_lowout] = -1
test['4_out_fee'][fee4_lowout] = -1

In [1247]:
test = test.drop(['former_complaint_fee', 'former_complaint_num'], axis=1)

In [1248]:
test.to_csv('test_pre.csv', index=False)

In [1249]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 743974 entries, 0 to 743973
Data columns (total 54 columns):
service_type            743974 non-null int64
online_time             743974 non-null int64
1_total_fee             743974 non-null float64
2_total_fee             743974 non-null float64
3_total_fee             743974 non-null float64
4_total_fee             743974 non-null float64
month_traffic           743974 non-null float64
contract_time           743974 non-null int64
pay_times               743974 non-null int64
pay_num                 743974 non-null float64
last_month_traffic      743974 non-null float64
local_trafffic_month    743974 non-null float64
local_caller_time       743974 non-null float64
service1_caller_time    743974 non-null float64
service2_caller_time    743974 non-null float64
gender                  743974 non-null object
age                     743974 non-null object
current_service         743974 non-null int64
user_id                 743974 non-nu

In [1250]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200000 entries, 0 to 199999
Data columns (total 53 columns):
service_type            200000 non-null int64
online_time             200000 non-null int64
1_total_fee             200000 non-null float64
2_total_fee             200000 non-null float64
3_total_fee             200000 non-null float64
4_total_fee             200000 non-null float64
month_traffic           200000 non-null float64
contract_time           200000 non-null int64
pay_times               200000 non-null int64
pay_num                 200000 non-null float64
last_month_traffic      200000 non-null float64
local_trafffic_month    200000 non-null float64
local_caller_time       200000 non-null float64
service1_caller_time    200000 non-null float64
service2_caller_time    200000 non-null float64
gender                  200000 non-null int64
age                     200000 non-null int64
user_id                 200000 non-null object
fee_mean                200000 non-nul

In [184]:
# test['this_month_traffic'] = test['month_traffic'] - test['last_month_traffic']
# test['outland_traffic_month'] = test['this_month_traffic'] - test['local_trafffic_month']
# test['is_0'] = (test['month_traffic'] 
#                          == 0)&(test['last_month_traffic'] == 0)&(test['local_trafffic_month']==0)

In [185]:
# test['no_call'] = ((test['local_caller_time'] == 0)&
#                            (test['service1_caller_time'] == 0)&
#                            (test['service2_caller_time'] == 0)).astype(int)
# test['no_service_but_call'] = ((test['local_caller_time'] == 0)&
#                                         (test['service1_caller_time'] > 0)|
#                                         (test['service2_caller_time'] > 0)).astype(int)
# test['no_traffic'] = (test['last_month_traffic'] == test['month_traffic']).astype(int)

In [186]:
# test['out_traffic'] = test['month_traffic'] - test['local_trafffic_month']
# test['last_no_out'] = ((test['last_month_traffic'] > 0)& (test['service2_caller_time'] == 0)).astype(int)
# test['last_no_change_type'] = (test['last_month_traffic'] > 0).astype(int)
# test['now_right_fee'] = test['last_no_change_type']*test['2_total_fee']
# del test['last_no_change_type']

In [188]:
# test['12_fee'] = test['1_total_fee'] - test['2_total_fee']
# test['23_fee'] = test['2_total_fee'] - test['3_total_fee']
# test['34_fee'] = test['3_total_fee'] - test['4_total_fee']

In [200]:
# test['remain_fee'] = test['1_total_fee'] - test['month_traffic']*0.3
# test['base_fee'] = (test['month_traffic']-test['last_month_traffic'])*0.3+test['local_caller_time']*0.15
# test['last_remain_fee'] = test['2_total_fee'] - test['service1_caller_time']*0.15
# test['last2_remain_fee'] = test['3_total_fee'] - test['service2_caller_time']*0.15

In [192]:
# test['fee_to1'] = (test['1_total_fee']-test['fee_mean'])/test['fee_var']
# test['fee_to2'] = (test['2_total_fee']-test['fee_mean'])/test['fee_var']
# test['fee_to3'] = (test['3_total_fee']-test['fee_mean'])/test['fee_var']
# test['fee_to4'] = (test['4_total_fee']-test['fee_mean'])/test['fee_var']

In [117]:
# test['1_total_fee'] = test['1_total_fee'].apply(lambda x: np.sqrt(x))
# test['2_total_fee'] = test['2_total_fee'].apply(lambda x: np.sqrt(x))
# test['3_total_fee'] = test['3_total_fee'].apply(lambda x: np.sqrt(x))
# test['4_total_fee'] = test['4_total_fee'].apply(lambda x: np.sqrt(x))